In [30]:
from glob import glob
from tqdm import tqdm

import os
import re

pcm_path = '/wav2vec2/s-kr/fine-tune/dataset/kspon_12h_wav'
pcms = sorted(glob(os.path.join(pcm_path, '*.wav'), recursive=True))

In [31]:
print(len(pcms))
print(pcms[:3])

7630
['/wav2vec2/s-kr/fine-tune/dataset/kspon_12h_wav/KsponSpeech_000078.wav', '/wav2vec2/s-kr/fine-tune/dataset/kspon_12h_wav/KsponSpeech_000090.wav', '/wav2vec2/s-kr/fine-tune/dataset/kspon_12h_wav/KsponSpeech_000200.wav']


In [32]:
pcm = pcms[6]
txt = pcm.replace('.wav', '.txt')

with open(txt, "r", encoding='cp949') as f:
    raw_sentence = f.read()

print(txt)    
print(raw_sentence)

/wav2vec2/s-kr/fine-tune/dataset/kspon_12h_wav/KsponSpeech_000547.txt
다시 봐야 되나?



In [46]:
# ignore_regex1 = '[\,\?\.\!\-\;\:\"\n\/]'
# ignore_regex2 = "['o', 'n', 'u', 'b', 'l']"
double_space_pattern = r'\s\s+'

def special_filter(sentence, replace=None):
    NOISE = ['o', 'n', 'u', 'b', 'l']
    EXCEPT = ['?', '!', '.', '/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '=', ':', ';', ',', # frist line from ksponspeech
              '“', '”', "'", '"', "\n", '‘', '’', '>', 'ㆍ', '<', '·', '~', '`', '♤', '\\', '_', '…', '​', '」', '「',  # second line from ktalkspeech
              '₁', '♡', '．', '％', '！', '州', 'フ', '₂', '¸', '°', '¥', '}', '|']

    new_sentence = str()
    for idx, ch in enumerate(sentence):
        if idx + 1 < len(sentence) and ch in NOISE and sentence[idx + 1] == '/':
            continue

        if ch == '#':
            new_sentence += '샾'

        # elif ch == '%':
        #     if mode == 'phonetic':
        #         new_sentence += replace
        #     elif mode == 'spelling':
        #         new_sentence += '%'

        elif ch not in EXCEPT:
            new_sentence += ch
    
    new_sentence = re.sub(double_space_pattern, ' ', new_sentence.strip())
            
    return new_sentence

def bracket_filter(sentence):
    new_sentence = str()
    flag = False
    for ch in sentence:
        if ch == '(' and flag is False:
            flag = True
            continue
        if ch == '(' and flag is True:
            flag = False
            continue
        if ch != ')' and flag is False:
            new_sentence += ch
    return new_sentence

In [39]:
sentence = refine_text(raw_sentence)
sentence

'어떤 시베리아 등반했던 사람이 우리나라 사람인데 다리에 동상 걸려가지고 다리를 다 짤라+ 발가락을 다 짤라 냈어야 된대 그래서 발가락 수술하려고 시+ 한 이틀 후 사미* 이렇게 잡고'

In [47]:
for pcm in pcms[0:100]:
    txt = pcm.replace('.wav', '.txt')
    with open(txt, "r", encoding='cp949') as f:
        raw_sentence = f.read()
    # new_sentence = refine_text(raw_sentence)
    new_sentence = special_filter(bracket_filter(raw_sentence))
    print(pcm[-11:], new_sentence)

_000078.wav 경영학관데 연대 경제 뚫었어 편입으로 나 진짜 그래서 에타 학게가 있었어 탈홍합니다
_000090.wav 애들 막 못 막잖아
_000200.wav 닌텐도 주가가 엄청 오르고 그것 때문에 다시 회복이 많이 됐거든요
_000318.wav 아 그 배우가 너무 귀엽더라고 성격이
_000325.wav 돈이 있어야 택실 탄다니까
_000483.wav 근데 또
_000547.wav 다시 봐야 되나
_000559.wav 나도 근데 사실 많이 바껴서 그렇게 생각을 많이 하는 편이야 응 좀 시간이 뭐 두 달이나 시간 있는데 그거를 집에만 있고
_000571.wav 흠 나도 싫어하는데
_000578.wav 그 랜덤인데
_000645.wav 아 아 아
_000656.wav 맞아
_000823.wav 너 왜 막걸리 안 먹냐
_000927.wav 어 없었던 거 같애
_001147.wav 형 나 일곱 시에 일어나서
_001192.wav 너 지금 되게 외계인 같아
_001209.wav 근데 그때 애들이랑 갔을 떄는 한 명당 하나씩 시컸던 거 같애 큰 거 말고 그냥
_001437.wav 아 나 담요 갖고 올까 하다가 너무 짐이 많기도 하고 니가 그때 담요 당장 필요 없다 그래가지고 그냥 안 갖고 왔어 블라우스만 갖고 왔어
_001637.wav 어떤데 일 화 보니까
_001644.wav 도른은사람들한테 물 같은 존재인가 라고 생각했는데 물 같지는 않은 거야
_001680.wav 어 이제 오 기 넘어가야 돼
_001815.wav 그래서 일부러 그러는 건지 그렇더라구
_001971.wav 갈매기살 맛 나
_002107.wav 근데 어제 이제 얘기 들어봤는데 알고 보니까 이제
_002123.wav 아 그래 에이 그럼 아니겠지 외가 쪽도 괜찮아 아잉까 외가 쪽도 괜찮아 아 그래 아 그래
_002208.wav 걔는 서울로 상경해야 되는 거니까
_002249.wav 생윤도 재밌었는데 나 법정 생윤이었잖아
_002305.wav 그래도 몇 번 밥도 사 주고 해 주신 분이라서 응 너무 

In [27]:
from kspon_preprocess import sentence_filter

for idx, pcm in enumerate(pcms[:100]):
    txt = pcm.replace('.wav', '.txt')
    with open(txt, "r", encoding='cp949') as f:
        raw_sentence = f.read()
    new_sentence = sentence_filter(raw_sentence)
    print(pcm[-11:], new_sentence)

_000078.wav 경영학관데연대경제뚫었어편입으로나진짜그래서에타학게가있었어탈홍합니다
_000090.wav 애들막못막잖아
_000200.wav 닌텐도주가가엄청오르고그것때문에다시회복이많이됐거든요
_000318.wav 아그배우가너무귀엽더라고성격이
_000325.wav 돈이있어야택실탄다니까
_000483.wav 근데또
_000547.wav 다시봐야되나
_000559.wav 나도근데사실많이바껴서그렇게생각을많이하는편이야응좀시간이뭐두달이나시간있는데그거를집에만있고
_000571.wav 흠나도싫어하는데
_000578.wav 그랜덤인데
_000645.wav 아아아
_000656.wav 맞아
_000823.wav 너왜막걸리안먹냐
_000927.wav 어없었던거같애
_001147.wav 형나일곱시에일어나서
_001192.wav 너지금되게외계인같아
_001209.wav 근데그때애들이랑갔을떄는한명당하나씩시컸던거같애큰거말고그냥
_001437.wav 아나담요갖고올까하다가너무짐이많기도하고니가그때담요당장필요없다그래가지고그냥안갖고왔어블라우스만갖고왔어
_001637.wav 어떤데일화보니까
_001644.wav 도른은사람들한테물같은존재인가라고생각했는데물같지는않은거야
_001680.wav 어이제오기넘어가야돼
_001815.wav 그래서일부러그러는건지그렇더라구
_001971.wav 갈매기살맛나
_002107.wav 근데어제이제얘기들어봤는데알고보니까이제
_002123.wav 아그래에이그럼아니겠지외가쪽도괜찮아아잉까외가쪽도괜찮아아그래아그래
_002208.wav 걔는서울로상경해야되는거니까
_002249.wav 생윤도재밌었는데나법정생윤이었잖아
_002305.wav 그래도몇번밥도사주고해주신분이라서응너무고마워가지구
_002389.wav 몇달동안가게
_002410.wav 걔는영어를잘해가지고외국기업이나그런데서울올라와서원서넣고있드라대구사는데
_002435.wav 음그래뭐궁금한게뭔데
_002486.wav 내가제출한답안지를
_002652.wav 원래는이삼등급을걔가유지했었거든요
_002685.wav 그노트북은내가딱보니까이

In [24]:
pcm = pcms[0]
txt = pcm.replace('.pcm', '.txt')
with open(txt, "r", encoding='cp949') as f:
    raw_sentence = f.read()
raw_sentence

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 24: invalid start byte